In [ ]:
import os
import main_fte

%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
# Configurations for the optimisation.
root_dir = os.path.join("/Users/zico/OneDrive - University of Cape Town/msc/data/cheetah_videos")
# out_dir = os.path.join("/Users/zico/OneDrive - University of Cape Town/msc/data/analysed")
# If you are running the code remotely on the Linux i9.
# root_dir = os.path.join("/","data", "zico", "cheetah_videos")
# out_dir = os.path.join("/", "data", "zico", "analysed")
# data_path = "kinetic_dataset/2009_09_08/arabia/trial15"
# data_path = "kinetic_dataset/2009_09_07/arabia/trial06"
data_path = "kinetic_dataset/2009_09_08/shiraz/trial04"
# data_path = "kinetic_dataset/2009_09_08/shiraz/trial02"
# data_path = "kinetic_dataset/2009_09_07/shiraz/trial04"
# data_path = "kinetic_dataset/2009_09_11/shiraz/trial01"
# data_path = "kinetic_dataset/2009_09_11/shiraz/trial02"
# data_path = "2017_08_29/top/jules/run1_2"
start_frame = 108
end_frame = 147
dlc_thresh = 0.5

In [ ]:
# Run the optimisation.
# {"cam": 2, "frame": 10}, {"cam": 3, "frame": -51} trail02
main_fte.run(root_dir,
             data_path,
             start_frame,
             end_frame,
             dlc_thresh,
             sync_offset=[{
                 "cam": 2,
                 "frame": 23
             }, {
                 "cam": 3,
                 "frame": -48
             }],
             cam_est_idx=-1,
             hand_labeled=False,
             generate_reprojection_videos=True)


In [ ]:
 # Plot the 3D trajectory for visual inspection.
main_fte.plot_cheetah(root_dir, data_path, fte_type="fte")

In [ ]:
# Compare the 3D trajectory for two different reconstruct`ions.
main_fte.compare_cheetahs(root_dir, data_path, fte_type1="fte_0", fte_type2="fte")

In [ ]:
# Generate metrics.
main_fte.metrics(root_dir, data_path, start_frame, end_frame, dlc_thresh)

In [ ]:
from glob import glob
from lib.app import create_labeled_videos
video_paths = sorted(glob(os.path.join(root_dir, data_path, 'cam[1-9].mp4')))  # original vids should be in the parent dir
create_labeled_videos(video_paths,
                      out_dir=os.path.join(root_dir, data_path, "fte"),
                      draw_skeleton=True,
                      pcutoff=dlc_thresh)
